# Basic Convolutional Neural Network for classifying CIFAR-10 dataset

Written by: Joshua Chen

This Python notebook serves as a quick reference or simple tutorial on the CNN training pipeline.

## Import all necessary packages

In [1]:
import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os

## Define Hyperparameters

In [2]:
BATCH_SIZE = 64
EPOCHS = 10
LR = 1e-3
MODEL_SAVE_PATH = './Models'

if not os.path.exists(MODEL_SAVE_PATH):
    os.mkdir(MODEL_SAVE_PATH)
    
# Define image transforms and/or augmentations:
transform = transforms.Compose([transforms.Resize(32), 
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

## General Deep CV Neural Network Training Pipeline:

1) Import and normalize the dataset. 

2) Define neural network and import it.

3) Define Loss function and optimizer.

4) Train the network.

In [3]:
# Import and Normalize Dataset:
training = torchvision.datasets.CIFAR10(root='./cifar10Data', train=True, download=True, transform=transform)
trainLoader = DataLoader(training, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testing = torchvision.datasets.CIFAR10(root='./cifar10Data', train=False, download=True, transform=transform)
testLoader = DataLoader(testing, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Check to ensure GPU is available:
device = ("cuda" if torch.cuda.is_available() else "cpu")

## Import Custom CNN or use an existing architecture

In [12]:
# Import the Convolutional Neural Network:
import cnn_model
net = cnn_model.ConvNet().to(device)

In [11]:
# Initialize Premade Architecture if not using custom CNN:
from torchvision.models import resnet50, ResNet50_Weights
weights = ResNet50_Weights.DEFAULT
net = resnet50(weights=weights).to(device)

## Define Loss Function and Optimizer

In [12]:
# Define a Loss Function and Optimizer:
import torch.nn as nn
import torch.optim as optim

loss = nn.CrossEntropyLoss() # Softmax Loss
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9) # Stochastic Gradient Descent

## Train the Network

In [13]:
# Define the Train and Test functions:
def train(trainLoader, net, loss_fn, optimizer):
    size = len(trainLoader.dataset)
    net.train()
    for batch, (X,y) in enumerate(trainLoader):
        X, y = X.to(device), y.to(device) # Send training data to GPU
        
        # Compute Loss:
        pred = net(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation:
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1)*len(X)
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
            
def test(testLoader, net, loss_fn):
    size = len(testLoader.dataset)
    num_batches = len(testLoader)
    net.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in testLoader:
            X, y = X.to(device), y.to(device)
            pred = net(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches # Average Test Loss
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
for i in range(EPOCHS):
    print(f"Epoch {i+1}\n---------------------------------")
    train(trainLoader, net, loss, optimizer)
    test(testLoader, net, loss)
print("Finished Training!")

Epoch 1
---------------------------------
Loss: 13.725882 [   64/50000]
Loss: 6.597033 [ 6464/50000]
Loss: 5.677573 [12864/50000]
Loss: 3.349605 [19264/50000]
Loss: 2.068901 [25664/50000]
Loss: 1.505867 [32064/50000]
Loss: 1.540960 [38464/50000]
Loss: 1.378218 [44864/50000]
Test Error: 
 Accuracy: 63.8%, Avg loss: 1.109370 

Epoch 2
---------------------------------
Loss: 0.943730 [   64/50000]
Loss: 0.984301 [ 6464/50000]
Loss: 1.069256 [12864/50000]
Loss: 0.974396 [19264/50000]
Loss: 1.026370 [25664/50000]
Loss: 0.862955 [32064/50000]
Loss: 1.395810 [38464/50000]
Loss: 0.830054 [44864/50000]
Test Error: 
 Accuracy: 73.5%, Avg loss: 0.790166 

Epoch 3
---------------------------------
Loss: 0.646730 [   64/50000]
Loss: 0.818648 [ 6464/50000]
Loss: 0.504553 [12864/50000]
Loss: 0.705039 [19264/50000]
Loss: 0.685125 [25664/50000]
Loss: 0.751926 [32064/50000]
Loss: 0.556361 [38464/50000]
Loss: 0.720148 [44864/50000]
Test Error: 
 Accuracy: 77.9%, Avg loss: 0.685712 

Epoch 4
-------------

## Save Trained Model

In [15]:
torch.save(net, MODEL_SAVE_PATH + '/resnet50cifar10.pth')